In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
% matplotlib inline
vocab_size = 1848
max_len = 35

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pickle
encoding_train = pickle.load(open("encoded_train_features.pkl", "rb"))
encoding_test = pickle.load(open("encoded_test_features.pkl", "rb"))
embedding_matrix = pickle.load(open("embedding_matrix.pkl", "rb"))
word_to_idx = pickle.load(open("word_to_idx", "rb"))
train_descriptions = pickle.load(open("train_descriptions.pkl", "rb"))

In [0]:
from keras.preprocessing.sequence import pad_sequences

In [0]:
# generator remembers the state of the function in the previous call
def data_generator(train_descriptions, encoding_train, word_to_idx, max_len, batch_size):
    X1, X2, y = [], [], []
    n = 0
    while True:
        for key, desc_list in train_descriptions.items():
            n += 1
            photo = encoding_train[key]
            for desc in desc_list:
                seq = [word_to_idx[word] for word in desc.split() if word in word_to_idx]
                for i in range(1, len(seq)):
                    xi = seq[0:i]
                    yi = seq[i]
                    xi = pad_sequences([xi], maxlen = max_len, value = 0, padding= 'post')[0] # value=0 denotes the padding word
                    yi = to_categorical([yi], num_classes = vocab_size)[0] # one hot encoding of the word corresponding to y
                    
                    # make a mini batch
                    X1.append(photo)
                    X2.append(xi)
                    y.append(yi)
                    
                    if n == batch_size:
                        yield ([np.array(X1), np.array(X2)], np.array(y))
                        
                        # prepare for next iteration
                        X1, X2, y = [],[],[]
                        n = 0

In [0]:
from keras.layers import Input, Dense, Dropout, Embedding, LSTM, Add
from keras.models import Model

In [11]:
input_img_features = Input(shape=(2048,))
inp_img1 = Dropout(0.3)(input_img_features)
inp_img2 = Dense(256, activation='relu')(inp_img1)




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.



In [12]:
input_captions = Input(shape=(35,))
inp_cap1 = Embedding(input_dim = vocab_size, output_dim = 50, mask_zero = True)(input_captions)
inp_cap2 = Dropout(0.3)(inp_cap1)
inp_cap3 = LSTM(256)(inp_cap2)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
decoder1 = Add()([inp_img2, inp_cap3])
decoder2 = Dense(256, activation='relu')(decoder1)
outputs = Dense(vocab_size, activation = 'softmax')(decoder2)

In [0]:
model = Model(inputs = [input_img_features, input_captions], outputs=outputs) # combined model

In [15]:
model.layers[2].set_weights([embedding_matrix])
model.layers[2].trainable = False

In [16]:
model.compile(loss = "categorical_crossentropy", optimizer= "adam")

In [0]:
epochs = 20
batch_size = 3
steps = len(train_descriptions)//batch_size
from keras.utils.np_utils import to_categorical

In [18]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [0]:
def train():
    for i in range(epochs):
        generator = data_generator(train_descriptions, encoding_train, word_to_idx, max_len, batch_size) # generator instance
        model.fit_generator(generator, epochs = 1, steps_per_epoch = steps, verbose=1)
        model.save('gdrive/My Drive/model_' + str(i) + '.h5')

In [20]:
train()



Epoch 1/1
2000/2000 [==============================] - 237s 118ms/step - loss: 4.3023
Epoch 1/1
2000/2000 [==============================] - 235s 117ms/step - loss: 3.5813
Epoch 1/1
2000/2000 [==============================] - 235s 118ms/step - loss: 3.3214
Epoch 1/1
2000/2000 [==============================] - 236s 118ms/step - loss: 3.1630
Epoch 1/1
2000/2000 [==============================] - 237s 119ms/step - loss: 3.0499
Epoch 1/1
2000/2000 [==============================] - 235s 118ms/step - loss: 2.9649
Epoch 1/1
2000/2000 [==============================] - 236s 118ms/step - loss: 2.8962
Epoch 1/1
2000/2000 [==============================] - 234s 117ms/step - loss: 2.8387
Epoch 1/1
2000/2000 [==============================] - 235s 118ms/step - loss: 2.7950
Epoch 1/1
2000/2000 [==============================] - 235s 118ms/step - loss: 2.7527
Epoch 1/1
2000/2000 [==============================] - 236s 118ms/step - loss: 2.7147
Epoch 1/1
2000/2000 [==============================]